In [1]:
# Faster RCNN network to find and classify cells in images with Pytorch


# Imports
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.utils as utils
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
import time
import copy
import random
import math
import cv2
import pandas as pd


ImportError: cannot import name 'Path' from 'os' (/Users/deniz/opt/anaconda3/envs/comp411/lib/python3.7/os.py)

In [3]:
# read in the csv file as a pandas dataframe
df_train = pd.read_csv("QC/df_train.csv")
df_valid = pd.read_csv("QC/df_valid.csv")
df_test = pd.read_csv("QC/df_test.csv")

In [4]:
# Define the dataset class where the inputs are:
# df: a dataframe that contains the image id, classification and xmax, xmin, ymin, ymax for the bounding box
# image_dir and transforms
class CellDataset(Dataset):
    def __init__(self, df, image_dir, transforms=None):
        super().__init__()
        self.df = df
        self.image_ids = self.df["image_id"].unique()
        self.image_dir = os.path(image_dir)
        self.transforms = transforms
        
    def __len__(self):
        return self.image_ids.shape[0]
    
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        records = self.df[self.df["image_id"] == image_id]

        # get the image
        image_name = image_id + ".png"
        image = Image.open(os.path.join(self.image_dir, image_name)).convert("RGB")
        image = transforms.ToTensor()(image)

        # get the bounding box coordinates
        boxes = records[["xmin", "ymin", "xmax", "ymax"]].values
        boxes = torch.as_tensor(boxes, dtype=torch.int64)


        # get the classification
        labels = torch.tensor(records["super_classification"].values, dtype=torch.int64)
        

        # create a target dictionary
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = torch.tensor([idx])


        # apply the transformations
        if self.transforms is not None:
            image = self.transforms(image)
            
        return image, target, image_id
    


In [ ]:
# Define the Faster RCNN model
model = models.detection.fasterrcnn_resnet50_fpn_v2(pretrained=True, progress=True, num_classes=5, pretrained_backbone=True, 
                                trainable_backbone_layers=0, image_mean=[0.485, 0.456, 0.406], image_std=[0.229, 0.224, 0.225])

# Train the model for 5 epochs
num_epochs = 5

# Define the optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Define the learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Define the loss function
loss_func = nn.CrossEntropyLoss()

# Define the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Define the training and validation datasets
train_dataset = CellDataset(df_train, "QC/images/", transforms=transforms.Compose([transforms.ToTensor()]))
valid_dataset = CellDataset(df_valid, "QC/images/", transforms=transforms.Compose([transforms.ToTensor()]))
test_dataset = CellDataset(df_test, "QC/images/", transforms=transforms.Compose([transforms.ToTensor()]))

# Define the training and validation dataloaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)

# Define the training loop
def train_model(model, loss_func, optimizer, lr_scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs - 1))
        print("-" * 10)

        # Each epoch has a training and validation phase
        for phase in ["train", "valid"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over the data
            for images, targets, image_ids in dataloaders[phase]:
                

